## Measuring shift on AuxTel Hysteresis run - 12-Dec-22

Craig Lage

In [ ]:
import sys, time, os, asyncio, glob

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u
from astropy.table import Table
from astroquery.astrometry_net import AstrometryNet
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.summit.utils import BestEffortIsr
from lsst.daf.butler import Butler
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig

In [ ]:
butler = Butler('/repo/LATISS', collections=["LATISS/raw/all", "LATISS/calib"])
best_effort_isr = BestEffortIsr()

In [ ]:
# Run one just to get it working
exposure = 2022121100275
dataId = {'exposure':exposure, 'detector':0, "instrument":'LATISS'}
best_effort_isr.doWrite = False  # Don't write to butler database
exp = best_effort_isr.getExposure(dataId, skipCosmics=False)

In [ ]:
charConfig = CharacterizeImageConfig()
charConfig.doMeasurePsf = False
charConfig.doApCorr = False
charConfig.doDeblend = False
charConfig.repair.doCosmicRay = True
charConfig.repair.doInterpolate = True   
charConfig.detection.minPixels = 500
charTask = CharacterizeImageTask(config=charConfig)

mData = exp.getMetadata()
charResult = charTask.run(exp)
sourceCatalog = charResult.sourceCat
maxFlux = np.nanmax(sourceCatalog['base_CircularApertureFlux_3_0_instFlux'])
selectBrightestSource = sourceCatalog['base_CircularApertureFlux_3_0_instFlux'] > maxFlux * 0.99
brightestSource = sourceCatalog.subset(selectBrightestSource)
brightestCentroid = (brightestSource['base_SdssCentroid_x'][0], \
                     brightestSource['base_SdssCentroid_y'][0])
brightCatalog = sourceCatalog.subset(sourceCatalog['base_CircularApertureFlux_3_0_instFlux'] > maxFlux * 0.001)
print(f"exposure:{exposure}. Found {len(sourceCatalog)} sources, {len(brightCatalog)} bright sources")
print(f"Brightest centroid at {brightestCentroid}")
pixelScale = exp.getWcs().getPixelScale().asArcseconds()
detectorCenter = (2048.5, 2000.5)
offset = np.subtract(brightestCentroid, detectorCenter) * pixelScale
print(f"Az_offset = {offset[0]} arcseconds, El_offset = {offset[1]} arcseconds")

In [ ]:
cat = brightCatalog
from matplotlib.colors import LogNorm
%matplotlib inline
Ncenter = (700, 900)
Nlength = 500.0
NcenterAzEl = (3200, 700)
Nlabel = 650.0
yShift = 150.0

plt.figure(figsize=(16,16))
plt.title(f"AuxTel Azimuth hysteresis, {exposure}", fontsize = 24)
arr = exp.image.array
arr = np.clip(arr, 1, 100000) # This image has some negative values, and this removes them
img = plt.imshow(arr, norm=LogNorm(vmin=1, vmax=1000),  interpolation='Nearest', cmap='gray')

plt.scatter(brightestCentroid[0],brightestCentroid[1]\
            ,color='red', marker='x', s=1000)

rotpa = Angle(mData['ROTPA']*u.deg)
plt.arrow(Ncenter[0],Ncenter[1], -Nlength*np.sin(rotpa), Nlength*np.cos(rotpa),\
    color='green', width = 20)
plt.text(Ncenter[0]-Nlabel*np.sin(rotpa),Ncenter[1]+Nlabel*np.cos(rotpa), 'N', \
    color='green', fontsize=12, weight='bold')
plt.arrow(Ncenter[0],Ncenter[1], Nlength*np.cos(rotpa), Nlength*np.sin(rotpa),\
    color='green', width = 20)
plt.text(Ncenter[0]+Nlabel*np.cos(rotpa),Ncenter[1]+Nlabel*np.sin(rotpa), 'E', \
    color='green', fontsize=12, weight='bold')

el = Angle(mData['ELSTART'] * u.deg)
az = Angle(mData['AZSTART'] * u.deg)
dec = Angle(mData['DECSTART'] * u.deg)
sinTheta =  np.cos(AUXTEL_LOCATION.lat) / np.cos(dec) * np.sin(az)
theta = Angle(np.arcsin(sinTheta))
rotAzEl = rotpa - theta - Angle(90.0 * u.deg)   
print(theta.deg, rotpa.deg, rotAzEl.deg)
plt.arrow(NcenterAzEl[0],NcenterAzEl[1], -Nlength*np.sin(rotAzEl), Nlength*np.cos(rotAzEl),\
    color='cyan', width = 20)
plt.text(NcenterAzEl[0]-Nlabel*np.sin(rotAzEl),NcenterAzEl[1]+Nlabel*np.cos(rotAzEl), 'EL', \
    color='cyan', fontsize=12, weight='bold')
plt.arrow(NcenterAzEl[0],NcenterAzEl[1], Nlength*np.cos(rotAzEl), Nlength*np.sin(rotAzEl),\
    color='cyan', width = 20)
plt.text(NcenterAzEl[0]+Nlabel*np.cos(rotAzEl),NcenterAzEl[1]+Nlabel*np.sin(rotAzEl), 'AZ', \
    color='cyan', fontsize=12, weight='bold')

plt.text(brightestCentroid[0]-200.0,brightestCentroid[1] - 200.0,  \
    f"Az_offset = {offset[0]:.2f} arcseconds, El_offset = {offset[1]:.2f} arcseconds", \
         color='yellow', fontsize=12, weight='bold')

plt.savefig(f"/home/craiglagegit/DATA/AuxTel_Azimuth_Hysteresis_{exposure}_12Dec22.pdf")

In [ ]:
# Now run them all


outfile = "/home/craiglagegit/DATA/AuxTel_Azimuth_Hysteresis_12Dec22.txt"
file = open(outfile, "w")
file.write("Exposure\tAz(degrees)\tEl(degrees)\tDeltaAz(sec)\tDeltaEl(sec)\n")
for exposure in range(2022121100232, 2022121100319):
    dataId = {'exposure':exposure, 'detector':0, "instrument":'LATISS'}
    best_effort_isr.doWrite = False  # Don't write to butler database
    exp = best_effort_isr.getExposure(dataId, skipCosmics=False)
    mData = exp.getMetadata()
    az = mData["AZSTART"]
    el = mData["ELSTART"]
    charResult = charTask.run(exp)
    sourceCatalog = charResult.sourceCat
    maxFlux = np.nanmax(sourceCatalog['base_CircularApertureFlux_3_0_instFlux'])
    selectBrightestSource = sourceCatalog['base_CircularApertureFlux_3_0_instFlux'] > maxFlux * 0.99
    brightestSource = sourceCatalog.subset(selectBrightestSource)
    brightestCentroid = (brightestSource['base_SdssCentroid_x'][0], \
                         brightestSource['base_SdssCentroid_y'][0])
    brightCatalog = sourceCatalog.subset(sourceCatalog['base_CircularApertureFlux_3_0_instFlux'] > maxFlux * 0.001)
    print(f"exposure:{exposure}. Found {len(sourceCatalog)} sources, {len(brightCatalog)} bright sources")
    print(f"Brightest centroid at {brightestCentroid}")
    pixelScale = exp.getWcs().getPixelScale().asArcseconds()
    detectorCenter = (2048.5, 2000.5)
    offset = np.subtract(brightestCentroid, detectorCenter) * pixelScale
    print(f"Az_offset = {offset[0]} arcseconds, El_offset = {offset[1]} arcseconds")
    file.write(f"{exposure}\t{az:.4f}\t{el:.4f}\t{offset[0]:.2f}\t{offset[1]:.2f}\n")
file.close()

In [ ]:
# Now read in the data and plot it.
outfile = "/home/craiglagegit/DATA/AuxTel_Azimuth_Hysteresis_12Dec22.txt"
data = np.loadtxt(outfile, skiprows=1)
exposures = []
Azs = []
Els = []
deltaAzs = []
deltaEls = []
for j in range(data.shape[0]):
    [exposure, Az, El, deltaAz, deltaEl] = data[j]
    Azs.append(Az)
    Els.append(El)
    deltaAzs.append(deltaAz)
    deltaEls.append(deltaEl)
    exposures.append(exposure - 2022121100000)


In [ ]:
plt.figure(figsize=(8, 10))
plt.subplots_adjust(hspace = 0.5)
plt.subplot(2,1,1)
plt.title(f"AuxTel Azimuth Hysteresis DeltaAz", fontsize = 24)
plt.scatter(exposures, deltaAzs)
plt.xlabel("Sequence Number")
plt.ylabel("DeltaAz (arcseconds)")
#plt.xlim(273, 280)
plt.subplot(2,1,2)
plt.title(f"AuxTel Elevation Hysteresis DeltaEl", fontsize = 24)
plt.scatter(exposures, deltaEls)
plt.xlabel("Sequence Number")
plt.ylabel("DeltaEl (arcseconds)")
plt.savefig(f"/home/craiglagegit/DATA/AuxTel_Azimuth_Hysteresis_All_12Dec22.pdf")

In [ ]:

%matplotlib inline
plt.figure(figsize=(8, 10))
plt.subplots_adjust(hspace = 0.5)
plt.subplot(2,1,1)
plt.title(f"AuxTel Azimuth", fontsize = 24)
plt.scatter(exposures, Azs)
plt.xlabel("Sequence Number")
plt.ylabel("Az(degrees)")
plt.subplot(2,1,2)
plt.title(f"AuxTel Elevation ", fontsize = 24)
plt.scatter(exposures, Els)
plt.xlabel("Sequence Number")
plt.ylabel("El(degrees)")
plt.savefig(f"/home/craiglagegit/DATA/AuxTel_Azimuth_Hysteresis_Images_12Dec22.pdf")

In [ ]:
Az_pos = Azs[24:42] + Azs[66:84]
DeltaAz_pos = deltaAzs[24:42] + deltaAzs[66:84]
Az_neg = Azs[3:24] + Azs[45:66]
DeltaAz_neg = deltaAzs[3:24] + deltaAzs[45:66]

In [ ]:
xaxis = np.linspace(-42, 4, 100)
fit_pos = np.polyfit(Az_pos, DeltaAz_pos, 1)
fit_neg = np.polyfit(Az_neg, DeltaAz_neg, 1)
yaxis_pos = np.polyval(fit_pos, xaxis)
yaxis_neg = np.polyval(fit_neg, xaxis)
%matplotlib inline
plt.figure(figsize=(8, 8))
plt.subplots_adjust(hspace = 0.5)
plt.subplot(1,1,1)
plt.title(f"AuxTel Azimuth", fontsize = 24)
plt.scatter(Az_pos, DeltaAz_pos, color='red', label = "Positive")
plt.plot(xaxis, yaxis_pos, ls='--', color='red')
plt.text(-25, -1.0, f"Slope={fit_pos[0]:.2f} arcsec/deg, Intercept={fit_pos[1]:.2f} arcsec", color='red')
plt.scatter(Az_neg, DeltaAz_neg, color='blue', label = "Negative")
plt.plot(xaxis, yaxis_neg, ls='--', color='blue')
plt.text(-40, -5.0, f"Slope={fit_neg[0]:.2f} arcsec/deg, Intercept={fit_neg[1]:.2f} arcsec", color='blue')
plt.xlabel("Az(degrees)")
plt.ylabel("DeltaAz(arcseconds)")
plt.legend()
plt.savefig(f"/home/craiglagegit/DATA/AuxTel_Azimuth_Hysteresis_Summary_12Dec22.pdf")

In [ ]:
Az_pos = Azs[24:45] + Azs[66:87]
DeltaAz_pos = deltaAzs[24:45] + deltaAzs[66:87]
Az_neg = Azs[0:24] + Azs[45:69]
DeltaAz_neg = deltaAzs[0:24] + deltaAzs[45:69]

In [ ]:
xaxis = np.linspace(-42, 15, 100)
fit_pos = np.polyfit(Az_pos, DeltaAz_pos, 1)
fit_neg = np.polyfit(Az_neg, DeltaAz_neg, 1)
yaxis_pos = np.polyval(fit_pos, xaxis)
yaxis_neg = np.polyval(fit_neg, xaxis)
%matplotlib inline
plt.figure(figsize=(8, 8))
plt.subplots_adjust(hspace = 0.5)
plt.subplot(1,1,1)
plt.title(f"AuxTel Azimuth", fontsize = 24)
plt.scatter(Az_pos, DeltaAz_pos, color='red', label = "Positive")
plt.plot(xaxis, yaxis_pos, ls='--', color='red')
plt.text(-25, -1.0, f"Slope={fit_pos[0]:.2f} arcsec/deg, Intercept={fit_pos[1]:.2f} arcsec", color='red')
plt.scatter(Az_neg, DeltaAz_neg, color='blue', label = "Negative")
plt.plot(xaxis, yaxis_neg, ls='--', color='blue')
plt.text(-40, -5.0, f"Slope={fit_neg[0]:.2f} arcsec/deg, Intercept={fit_neg[1]:.2f} arcsec", color='blue')
plt.xlabel("Az(degrees)")
plt.ylabel("DeltaAz(arcseconds)")
plt.legend()
plt.savefig(f"/home/craiglagegit/DATA/AuxTel_Azimuth_Hysteresis_Summary_14Dec22.pdf")

In [ ]:
for i in range(len(exposures)):
    print(i, exposures[i], Azs[i])